In [1]:
import pandas as pd
from pandas import DataFrame
from os import listdir
from os.path import isfile, join
mypath = 'C:/Users/User/Desktop/Cliodinámica/BBDD/CENSO/Personas/' #Ubicación de la base de datos y los archivos de etiqueta
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))] #Lectura de todos los archivos en la carpeta indicada

In [2]:
#Función para eliminar acentos y dejar todo con mayusculas
def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
        ("ä", "a"),
        ("ë", "e"),
        ("ï", "i"),
        ("ö", "o"),
        ("ü", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
        s = s.upper()
    return s

In [3]:
loc_ind = []
tags = []
df_desc = pd.read_csv(mypath + "Var_Des.csv",sep=';')
total_tag = list(df_desc['NOMBRE VARIABLE EN BBDD'])
aux = 0
for files in onlyfiles: #Por cada archivo en carpeta, cuyo listado es onlyfiles
    if files.startswith("etiquetas"): #Si comienza por la palabra etiquetas
        tag_files = normalize(files[10:-4]) #Extraer información importante, en este caso el código de la variable
        for des_tag in total_tag:
            if des_tag == tag_files:
                loc_ind.append(aux) #Identifica la ubicación del archivo que nos interesa en onlyfiles y se agrega a una lista
                tags.append(normalize(tag_files)) #Agrega el nombre de la variable en mayusculas en una lista
        aux += 1
print(tags)

['P07', 'P08', 'P10', 'P11', 'P12', 'P12A_TRAMO', 'P13', 'P15', 'P16', 'P16A', 'P16A_OTRO', 'P17']


In [4]:
#Read datab
df_censo = pd.read_csv(mypath+'Microdato_Censo2017-Personas.csv',sep=';') #Lectura de la base de datos de CENSO
#Crea un diccionario con varios dataframes, compuesto por la comuna y la variable que nos interesa
#De acerudo a la lista 'tags' creada anteriormente
d = {'df_' + i: df_censo[['COMUNA', i]] for i in tags} 

In [5]:
largo = list(range(0,len(tags)))
print(largo)
#Crea un diccionario con todas las etiquetas extraídas desde la carpeta
print(onlyfiles[loc_ind[1]])
et = {'et_' + tags[i]: pd.read_csv(mypath + onlyfiles[loc_ind[i]],sep=';') for i in largo} 

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
etiquetas_p08.csv


In [28]:
df_censo.groupby(["COMUNA","P10"])["P10"].sum()

COMUNA  P10
1101    1      179107
        2        9752
        3       12939
        4        5068
        99     188595
                ...  
16305   1       11052
        2         160
        3        1140
        4          12
        99       8712
Name: P10, Length: 1712, dtype: int64

In [29]:
df_name = ['df_'+ name for name in tags]
et_name = ['et_'+ name for name in tags]
for i in largo:    
    val = list(et[et_name[i]]['valor'])
    glo = list(et[et_name[i]]['glosa']+ ' ' + tags[i])
    d[df_name[i]][tags[i]].replace(val,glo,inplace=True)

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [53]:
d["df_P16A_OTRO"].groupby(["COMUNA","P16A_OTRO"])["P16A_OTRO"].count().filter(like="1101", axis=0)

COMUNA  P16A_OTRO                            
1101    Afrodescendiente P16A_OTRO                  308
        Changos P16A_OTRO                            72
        Chonos P16A_OTRO                              3
        Huilliche P16A_OTRO                          43
        Missing P16A_OTRO                          4183
        No aplica P16A_OTRO                      185532
        Ona P16A_OTRO                                 5
        Pehuenche P16A_OTRO                           8
        Picunche P16A_OTRO                            3
        Pueblo no declarado P16A_OTRO               856
        Pueblos de América Latina P16A_OTRO         421
        Pueblos del resto del mundo P16A_OTRO        32
        Tehuelches P16A_OTRO                          2
11101   Afrodescendiente P16A_OTRO                    8
        Chonos P16A_OTRO                             11
        Huilliche P16A_OTRO                         616
        Missing P16A_OTRO                          1026
  

In [18]:
mat_desc = []
for i in largo:
    desc_list =[]
    name_var = df_desc['NOMBRE VARIABLE EN BBDD'][i]
    var = df_desc['VARIABLE'][i]
    subdim = df_desc['SUBDIMENSIÓN'][i]
    dim = df_desc['DIMENSIÓN'][i]
    fuente = "CENSO PERSONAS"
    year = "2017"
    desc = df_desc['DESCRIPCIÓN'][i]
    val = list(et[et_name[i]]['glosa'])
    alc = "COMUNA"
    for etiqueta in val:
        desc_list = [dim, subdim, var, etiqueta, desc, name_var, fuente, year, alc]
        mat_desc.append(desc_list)

In [19]:
dat_name = ['dat_'+ name for name in tags]
dat = {'dat_' + tags[i]: d[df_name[i]].groupby(['COMUNA',tags[i]]).size().unstack(fill_value=0) for i in largo}

In [20]:
df = pd.concat(dat.values(),axis = 1)
df.columns[5].startswith("H")
for columna in df.columns:
    if columna.startswith("Missing") or columna.startswith("No aplica"):
        df.drop(columna, inplace = True, axis = 1)

In [21]:
cod_com = 'Test_CUT.csv'
#Read data
print('Leyendo archivos complementarios...')
df_comuna = pd.read_csv(mypath+cod_com,sep=";")

Leyendo archivos complementarios...


In [22]:
def comunal(DF,df_comuna):   
    print("Estandarizando nombres de comunas...")
    comuna = "COMUNA"
    DF = DF.reset_index()
    DF[comuna] = DF[comuna].apply(str)
    DF[comuna] = [normalize(x) for x in DF[comuna]]
    df_comuna["cod_com"] =df_comuna["cod_com"].apply(str) 
    codigo, nombre = df_comuna.columns
    codigo = list(df_comuna[codigo])
    nombre = list(df_comuna[nombre])
    filas, columnas = DF.shape
    print("Reemplazando nombres por códigos únicos territoriales...")
    DF[comuna].replace("CALERA","LA CALERA",inplace=True)
    DF[comuna].replace("PAIGUANO","PAIHUANO",inplace=True)
    DF[comuna].replace(nombre,codigo,inplace=True)
    df_comuna = df_comuna.set_index('cod_com')
    DF = DF.set_index(comuna)
    DF = pd.concat([df_comuna,DF.reindex(df_comuna.index)],axis=1)
    DF = DF.drop(columns = "nom_com")
    DF = DF.where(pd.notnull(DF), None)
    print("Listo!")
    print("  ")
    return DF
df = comunal(df,df_comuna)

Estandarizando nombres de comunas...
Reemplazando nombres por códigos únicos territoriales...
Listo!
  


In [23]:
list_data = []
for var in (tags):
    for name_col in df.columns:
        for x in mat_desc:
            if var == name_col[-len(var):] and name_col[:-(len(var)+1)]==x[3]:
                data = (x + list(df[name_col]))
                list_data.append(data)  
aux = len(list_data)
for i in range(aux):
    for j in range(8):
        list_data[i][j] = (list_data[i][j]).upper()

In [24]:
path_matrix = 'C:/Users/User/Desktop/Cliodinámica/BBDD/CENSO/Personas/'
only_matrix = [f for f in listdir(path_matrix) if isfile(join(path_matrix, f))]
df_matrix = pd.read_csv(path_matrix+'Matriz_Consulta.csv',sep=';')
print(len(list_data))
df_nueva = df_matrix
for lista in list_data:
    df_nueva = df_nueva.append(pd.Series(lista, index = df_nueva.columns), ignore_index = True)


122


In [38]:
df_nueva.to_csv("Matriz_consulta_N.csv") 

In [25]:
df_nueva.to_excel("Matriz_consulta_N.xlsx", encoding='utf8')